<a href="https://colab.research.google.com/github/kamyarmrd/BML-project-OAI/blob/main/WholeOAIMRIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing libraries
!pip install SimpleITK
!pip install pydicom
!pip install dicom2nifti

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
# Importing libraries
%matplotlib inline
import glob
import cv2
import random
import os
import shutil
import pydicom
import nibabel as nib
import numpy as np
import SimpleITK as sitk
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
import os
import shutil
import re  # Regular expression library
import pydicom


# specify the source directory and destination directory
root_dir = '/content/drive/MyDrive/FNIH'  #please change this path to the folder of All baseline MRI images of OAI
new_dir = '/content/drive/MyDrive/SAG_OAIMRIsBaseline' #please create a new folder first and them copy its path here

# Create the new directory if it doesn't exist
os.makedirs(new_dir, exist_ok=True)

# Iterate over all subdirectories and files in root_dir
for subdir, dirs, files in os.walk(root_dir):
    # Split the current directory path into components
    path_components = os.path.normpath(subdir).split(os.sep)

        # Get the patient ID from the name of the fifth subdirectory in the path
    patient_id = path_components[4]

    # Initialize a counter
    counter = 0
        # Iterate over all files in the current subdirectory
    for filename in files:
            # Construct the full path to the file
        file_path = os.path.join(subdir, filename)

            # Check if this is a DICOM file (i.e., it has no file extension)
        if os.path.isfile(file_path) and not os.path.splitext(file_path)[1] and not filename.endswith('.jpg'):
                # Read the DICOM file
            ds = pydicom.dcmread(file_path, force=True)

                # Convert the DICOM dataset to a string
            ds_str = str(ds)

                # Determine the view position and side based on the DICOM dataset
            view_position = 'SAG_IW' if 'SAG_IW' in ds_str else 'Unknown'
            side = 'LEFT' if 'LEFT' in ds_str else ('RIGHT' if 'RIGHT' in ds_str else 'Unknown')

                # Extract slice number from filename and convert it to integer


            slice_number = int(re.search(r'\d+', filename).group())

                # Construct a new name for the file based on its metadata and original name
            new_name = f"{patient_id}_{view_position}_{side}_{slice_number}_{counter}"

                # Construct the full path to the new file
            new_path = os.path.join(new_dir, new_name)

                # Copy the file to new directory if view_position is 'SAG_IW'
            if view_position == 'SAG_IW':
                shutil.copy2(file_path, new_path)
                counter += 1

In [ ]:
#----------Convert DICOM to NIFTI and move them to a new folder------------------------

import os
import numpy as np
import nibabel as nib
import pydicom

def dicom2nifti(input_dir):
    # Create a new directory at the location of the input directory
    output_dir = os.path.join(os.path.dirname(input_dir), 'nifti_MRIs00')
    os.makedirs(output_dir, exist_ok=True)

    # Create a dictionary to store the DICOM files for each subject_id and side
    dicom_files = {}

    # Iterate over all files in input_dir
    for filename in os.listdir(input_dir):
        # Check if 'RIGHT' or 'LEFT' is in the filename
        if 'RIGHT' in filename or 'LEFT' in filename:
            # Extract the subject_id, side, and slice_number from the filename
            parts = filename.split('_')
            subject_id = parts[0]
            side = parts[3]
            slice_number = parts[-1]

            # Add the file to the dictionary
            if (subject_id, side) not in dicom_files:
                dicom_files[(subject_id, side)] = []
            dicom_files[(subject_id, side)].append((int(slice_number), os.path.join(input_dir, filename)))

    # Convert the lists of DICOM files to NIFTI format
    for (subject_id, side), files in dicom_files.items():
        # Sort the files based on the slice number
        files.sort()

        # Read the DICOM files
        slices = [pydicom.dcmread(f) for _, f in files]

        # Get the pixel data from the slices and stack them along a new first axis
        img = np.stack([s.pixel_array for s in slices], axis=2)

        # Create a NIFTI image from the pixel data
        img_nifti = nib.Nifti1Image(img, np.eye(4))

        # Construct the path to save the new NIFTI file
        new_file_path = os.path.join(output_dir, f"{subject_id}_{side}.nii")

        # Save the NIFTI image
        nib.save(img_nifti, new_file_path)

dicom2nifti('/content/drive/MyDrive/SAG_OAIMRIsBaseline') #please change this to the path for output folder of the previous bit



In [ ]:
#----------------------------------------left-right orientation------------------------------------
import os
import numpy as np
import nibabel as nib
import shutil

def flip_images(input_dir):
    # Create a new directory at the location of the input directory
    output_dir = os.path.join(os.path.dirname(input_dir), 'orientedMRIs00')
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over all files in input_dir
    for filename in os.listdir(input_dir):
        # Construct the path to the file
        file_path = os.path.join(input_dir, filename)

        # Load the nifti image
        img = nib.load(file_path)

        # Check if 'RIGHT' is in the filename
        if 'RIGHT' in filename:
            # Flip the image data along the third dimension (slices)
            flipped_data = img.get_fdata()[:, :, ::-1]

            # Convert the data to 16-bit integers
            flipped_data = flipped_data.astype(np.int16)

            # Create a new nifti image with the flipped data and original affine matrix
            flipped_img = nib.Nifti1Image(flipped_data, img.affine)

            # Construct the path to save the new nifti file
            new_file_path = os.path.join(output_dir, filename)

            # Save the new nifti image in output_dir
            nib.save(flipped_img, new_file_path)
        else:
            # Construct the path to save the original nifti file in output_dir
            new_file_path = os.path.join(output_dir, filename)

            # Copy the original nifti image to the new directory without flipping it
            shutil.copy(file_path, new_file_path)

flip_images('/content/drive/MyDrive/nifti_MRIs00') #please change this to the path for output folder of the previous bit


In [ ]:
#-----------------------------change the images back to 2D slices and in tif format------
import os
import nibabel as nib
from PIL import Image

# specify your path
path = '/content/drive/MyDrive/orientedMRIs00' #please change this to the path for output folder of the previous bit

# iterate over all images in the directory
for image_name in os.listdir(path):
    # Construct the path to the file
    file_path = os.path.join(path, image_name)

    # Load the nifti image and convert it to a numpy array
    img_nifti = nib.load(file_path)
    image_array = img_nifti.get_fdata()

    # iterate over all slices in the 3D image
    for i in range(image_array.shape[2]):
        # get the i-th slice (add 1 to i because slice numbers start from 1)
        slice_2d = image_array[:, :, i]

        # normalize the slice to 0-255 range
        slice_2d = ((slice_2d - slice_2d.min()) * (1/(slice_2d.max() - slice_2d.min()) * 255)).astype('uint8')

        # create a PIL image from the 2D slice
        img_2d = Image.fromarray(slice_2d)

        # save the 2D image with a new name that includes the slice number (add 1 to i because slice numbers start from 1)
        img_2d.save(os.path.join(path, f'{image_name[:-4]}_{i+1}.tif'))

    # remove the original 3D nifti image
    os.remove(file_path)

In [ ]:
#---------------------categorize the slices into medial and lateral and add them and their names to numpy arrays----
import os
import numpy as np
import imageio

img_dir = '/content/drive/MyDrive/orientedMRIs00' #please change this to the same path you defined for the previous bit

# Initialize lists to store the medial and lateral images and masks
medial_img_list = []
lateral_img_list = []
medial_img_names = []  # New list for medial image names
lateral_img_names = []  # New list for lateral image names

# Get a list of image files in the directory
img_files = [f for f in os.listdir(img_dir) if f.endswith('.tif')]

# Sort the image files by their slice number
img_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

# Loop over the sorted image files
for img_file in img_files:
    # Set the path to the image file
    img_path = os.path.join(img_dir, img_file)

    # Load the image using imageio
    img = imageio.v2.imread(img_path)

    # Check if the slice number is less than 15 or greater than 21
    slice_number = int(img_file.split('_')[-1].split('.')[0])
    if slice_number < 18:
        # Add the image to the medial image list
        medial_img_list.append(img)
        medial_img_names.append(img_file)  # Save the name of the medial image
    elif slice_number >= 18:
        # Add the image to the lateral image list
        lateral_img_list.append(img)
        lateral_img_names.append(img_file)  # Save the name of the lateral image

# Convert the medial and lateral image lists to numpy arrays
medial_img_array = np.array(medial_img_list)
lateral_img_array = np.array(lateral_img_list)


In [ ]:
#----------------------save the numpy arrays to a new directory--------------------------
# specify the paths to the files where you want to save the numpy arrays
save_dir = '/content/drive/MyDrive/numpyimages00' #please create a new folder first and then copy its path here

# save the numpy arrays to the files
np.save(os.path.join(save_dir, 'medial_img_array.npy'), medial_img_array)
np.save(os.path.join(save_dir, 'lateral_img_array.npy'), lateral_img_array)
np.save(os.path.join(save_dir, 'medial_img_names.npy'), medial_img_names)
np.save(os.path.join(save_dir, 'lateral_img_names.npy'), lateral_img_names)